In [ ]:
pip install pyvisa

In [ ]:
pip install pyserial

In [4]:
import tkinter as tk
from tkinter import ttk, messagebox
import pyvisa # visa 장비 제어 라이브러리  
import serial # 직렬 포트 통신용 라이브러리 
import socket # TCP/IP 통신용 라이브러
import threading
import time

In [18]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())

('ASRL3::INSTR', 'ASRL5::INSTR')


# 전체 코드

In [6]:
class SCPIController:
    def __init__(self, root):
        self.root = root
        self.root.title("장비 제어 자동화")

        self.rm = pyvisa.ResourceManager()
        self.instrument = None
        self.serial = None
        self.socket = None
        self.connection_type = None
        self.is_measuring = False

        self.build_ui()

    def build_ui(self):
        self.txt_resource = tk.Entry(self.root, width=50)
        self.txt_resource.insert(0, "USB0::0x0B21::0x0025::INSTR") #usb로 연결된 scpi 장비 고유주소
        self.txt_resource.pack()

        self.connect_btn = tk.Button(self.root, text="Connect", command=self.connect)
        self.connect_btn.pack()

        self.disconnect_btn = tk.Button(self.root, text="Disconnect", command=self.disconnect, state=tk.DISABLED)
        self.disconnect_btn.pack()

        self.cmd_entry = tk.Entry(self.root, width=40)
        self.cmd_entry.pack()

        self.send_btn = tk.Button(self.root, text="Send Command", command=self.send_command, state=tk.DISABLED)
        self.send_btn.pack()

        self.output = tk.Text(self.root, height=10)
        self.output.pack()

    def connect(self):
        resource = self.txt_resource.get() #장비 주소 예: "USB0::0x2A8D::0x1301::INSTR" 또는 "COM3" 또는 "192.168.1.100:5025"
        try:
            if resource.startswith("USB"): #VISA 장비로 간주
                self.instrument = self.rm.open_resource(resource) # 함수로 장비에 연결 / 연결 객체를 instrument에 저장 
                self.connection_type = "USB"
            elif resource.startswith("COM"): # 시리얼 포트로 간주
                self.serial = serial.Serial(resource, baudrate=9600, timeout=2)
                self.connection_type = "SERIAL"
            elif ":" in resource:
                ip, port = resource.split(":") # "IP:Port" 형식
                self.socket = socket.create_connection((ip, int(port)), timeout=2)
                self.connection_type = "LAN"

            self.connect_btn.config(state=tk.DISABLED)
            self.disconnect_btn.config(state=tk.NORMAL)
            self.send_btn.config(state=tk.NORMAL)
            messagebox.showinfo("연결", "장비 연결 성공!")
        except Exception as e:
            messagebox.showerror("오류", f"연결 실패: {str(e)}")

    def disconnect(self):
        try:
            if self.connection_type == "USB" and self.instrument:
                self.instrument.close()
            elif self.connection_type == "SERIAL" and self.serial:
                self.serial.close()
            elif self.connection_type == "LAN" and self.socket:
                self.socket.close()

            self.instrument = self.serial = self.socket = None
            self.connection_type = None
            self.connect_btn.config(state=tk.NORMAL)
            self.disconnect_btn.config(state=tk.DISABLED)
            self.send_btn.config(state=tk.DISABLED)
            messagebox.showinfo("해제", "연결 해제됨")
        except Exception as e:
            messagebox.showerror("해제 오류", str(e))

    def send_command(self):
        cmd = self.cmd_entry.get().strip() #사용자가 입력한 명령어를 cmd변수에 저장
        if not cmd:
            return

        try:
            if self.connection_type == "USB":
                if cmd.endswith("?"): #?로 끝나니까 응답이 있는 쿼리 명령 하겠다?
                    response = self.instrument.query(cmd)
                    self.output.insert(tk.END, f"> {cmd}\n< {response}\n")
                else:
                    self.instrument.write(cmd)
                    self.output.insert(tk.END, f"> {cmd}\n")
            elif self.connection_type == "SERIAL": # 시리얼 장비는 응답을 한 줄 씩 보냄 
                self.serial.write((cmd + "\n").encode('utf-8'))
                if cmd.endswith("?"):
                    response = self.serial.readline().decode().strip()
                    self.output.insert(tk.END, f"> {cmd}\n< {response}\n")
            elif self.connection_type == "LAN":
                self.socket.sendall((cmd + "\n").encode('utf-8')) # utf-8추가
                if cmd.endswith("?"):
                    response = self.socket.recv(1024).decode().strip()
                    self.output.insert(tk.END, f"> {cmd}\n< {response}\n")
        except Exception as e:
            messagebox.showerror("전송 오류", str(e))

        self.cmd_entry.delete(0, tk.END) #명령어 초기화 -> 새로운 명령 입력 

# 실행
if __name__ == "__main__":
    root = tk.Tk()
    app = SCPIController(root)
    root.mainloop()

# 

# scpi명령어 

- *IDN?	장비 정보 요청 (제조사, 모델명 등)
- MEAS:VOLT?	전압 측정 요청
- VOLT 5	전압 5V로 설정
- CURR:RANG 10	전류 범위 10A로 설정
- OUTP ON	출력 켜기
- OUTP OFF	출력 끄기

*IDN?  
VOLT:RANG:AUTO ON  
MEAS:VOLT?  
OUTP ON

# 연습

In [34]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())

('ASRL3::INSTR', 'ASRL5::INSTR')


In [ ]:

rm = pyvisa.ResourceManager()
inst = rm.open_resource('GPIB0::10::INSTR')
inst.write('VOLT 220')
inst.write('FREQ 60')
inst.write('OUTP ON')
voltage = inst.query('VOLT?')
print(voltage)

In [35]:
import serial
ser = serial.Serial("COM3", 9600, timeout=2)
ser.write(b"TDVOLT?\r")  # APT 명령어 예시
print(ser.readline().decode())

In [37]:
ser = serial.Serial("COM5", baudrate=9600, stopbits=serial.STOPBITS_TWO, timeout=1)

# APT 장비는 \r (Carriage Return)을 기대할 수 있음
ser.write(b"TD?\r")
resp = ser.readline().decode().strip()
print("응답:", repr(resp))

응답: ''


# 시리얼 포트 

In [ ]:
import sys
import serial
import serial.tools.list_ports
import tkinter as tk
from tkinter import ttk, messagebox

class SerialApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Serial Command Sender")
        self.root.geometry("600x500")
        self.serial_port = None

        self.build_ui()

    def build_ui(self):
        # 설정 프레임
        serial_frame = ttk.LabelFrame(self.root, text="시리얼 포트 설정", padding=10)
        serial_frame.pack(fill="x", padx=10, pady=5)

        ttk.Label(serial_frame, text="포트 선택:").grid(row=0, column=0, sticky="e")
        self.port_combobox = ttk.Combobox(serial_frame, width=30)
        self.port_combobox.grid(row=0, column=1, sticky="w")
        self.update_ports()

        ttk.Label(serial_frame, text="Baudrate:").grid(row=1, column=0, sticky="e")
        self.baudrate_combobox = ttk.Combobox(serial_frame, values=["9600", "19200", "38400", "57600", "115200"])
        self.baudrate_combobox.set("115200")
        self.baudrate_combobox.grid(row=1, column=1, sticky="w")

        self.connect_button = ttk.Button(serial_frame, text="Connect", command=self.connect_serial)
        self.connect_button.grid(row=2, column=0, pady=5)
        self.disconnect_button = ttk.Button(serial_frame, text="Disconnect", command=self.disconnect_serial)
        self.disconnect_button.grid(row=2, column=1, pady=5)
        self.disconnect_button['state'] = tk.DISABLED

        self.status_label = ttk.Label(serial_frame, text="Status: Disconnected")
        self.status_label.grid(row=3, column=0, columnspan=2)

        # 명령어 입력 및 전송
        ttk.Label(self.root, text="Command to Send:").pack(anchor="w", padx=10, pady=(10, 0))
        self.command_entry = ttk.Entry(self.root, width=60)
        self.command_entry.pack(padx=10, pady=5, fill="x")

        self.send_button = ttk.Button(self.root, text="Send Command", command=self.send_command)
        self.send_button.pack(pady=5)
        self.send_button['state'] = tk.DISABLED

        # 응답 출력
        ttk.Label(self.root, text="Device Response:").pack(anchor="w", padx=10)
        self.response_text = tk.Text(self.root, height=15)
        self.response_text.pack(fill="both", expand=True, padx=10, pady=5)

    def update_ports(self):
        ports = serial.tools.list_ports.comports()
        self.port_combobox['values'] = [port.device for port in ports]
        if ports:
            self.port_combobox.set(ports[0].device)

    def connect_serial(self):
        try:
            port = self.port_combobox.get()
            baudrate = int(self.baudrate_combobox.get())

            self.serial_port = serial.Serial(
                port=port,
                baudrate=baudrate,
                timeout=1
            )
            self.status_label.config(text=f"Status: Connected to {port}")
            self.connect_button['state'] = tk.DISABLED
            self.disconnect_button['state'] = tk.NORMAL
            self.send_button['state'] = tk.NORMAL
            messagebox.showinfo("Connected", f"Connected to {port}")
        except Exception as e:
            messagebox.showerror("Connection Error", str(e))

    def disconnect_serial(self):
        if self.serial_port and self.serial_port.is_open:
            self.serial_port.close()
        self.status_label.config(text="Status: Disconnected")
        self.connect_button['state'] = tk.NORMAL
        self.disconnect_button['state'] = tk.DISABLED
        self.send_button['state'] = tk.DISABLED
        messagebox.showinfo("Disconnected", "Serial port disconnected.")

    def send_command(self, command):
        if self.serial_port and self.serial_port.is_open:
            full_command = command + '\n'  # LF = Line Feed = 0x0A
            try:
                self.serial_port.write(full_command.encode('ascii'))
                self.serial_port.flush()
                time.sleep(0.2)  # 기기 반응 대기 (필요시 조절)

                # ACK/NAK 응답 수신
                ack = self.serial_port.read(1)
                if ack == b'\x06':
                    self.received_data_text.insert(tk.END, f"Command OK: {command}\n")
                elif ack == b'\x15':
                    self.received_data_text.insert(tk.END, f"Command NAK/Error: {command}\n")
                else:
                    self.received_data_text.insert(tk.END, f"No ACK/NAK: Raw Response: {ack}\n")

            except Exception as e:
                messagebox.showerror("Command Error", str(e))

In [8]:
if __name__ == '__main__':
    root = tk.Tk()
    app = SerialApp(root)
    root.mainloop()

In [ ]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())

In [ ]:
#아마 com3는 내 마우스, com5로? 마우스 빼기
import serial

ser = serial.Serial(
    port="COM5",
    baudrate=9600,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1
)
ser.write(b"*IDN?\n")
print(ser.readline())

ASRL3::INSTR -> Error: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.


In [ ]:
# 시리얼
ser = serial.Serial(
    port='COM5',                  
    baudrate=9600,
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1
)

# 명령어 전송
ser.write(b'*IDN?\n')  # LF로 종료해야 함

# 응답 읽기
time.sleep(0.2)
response = ser.readline().decode(errors='ignore').strip()
print("응답:", response)


In [ ]:
#파이비자 
rm = pyvisa.ResourceManager()
inst = rm.open_resource('ASRL5::INSTR')  

inst.baud_rate = 9600
inst.data_bits = 8
inst.stop_bits = 1
inst.parity = pyvisa.constants.Parity.none
inst.write_termination = '\n'
inst.read_termination = '\n'
inst.timeout = 1000  # 1000ms

# 명령어 전송 및 응답 받기
print("IDN 응답:", inst.query("*IDN?"))